In [1]:
import mujoco
#@title Import packages for plotting and creating graphics
import time
import itertools
import numpy as np
from typing import Callable, NamedTuple, Optional, Union, List

# Graphics and plotting.
print('Installing mediapy:')
import mediapy as media
import matplotlib.pyplot as plt

import mujoco.viewer
from pathlib import Path


# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

model_dir = Path("/home/anton/master/mujoco_menagerie/universal_robots_ur5e")
model_xml = model_dir / "scene.xml"


xml = """
<mujoco>
  <worldbody>
    <light name="top" pos="0 0 1"/>
    <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_path(str(model_xml))
data = mujoco.MjData(model)

m = model
d = data

path = np.arange(-1, -2, -0.001)
i = 0
end = len(path)


with mujoco.viewer.launch_passive(m, d) as viewer:
  # Close the viewer automatically after 30 wall-seconds.
  start = time.time()
  while viewer.is_running(): #and time.time() - start < 30:
    step_start = time.time()

    # mj_step can be replaced with code that also evaluates
    # a policy and applies a control signal before stepping the physics.
    mujoco.mj_step(m, d)

    # Example modification of a viewer option: toggle contact points every two seconds.
    with viewer.lock():
      viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = int(d.time % 2)
    
    # if i < end:
    #   pos = path[i]
    #   print(path[i]) 
    #   d.qpos[1] = pos
    # i = i + 1

    # Pick up changes to the physics state, apply perturbations, update options from GUI.
    viewer.sync()


    # Rudimentary time keeping, will drift relative to wall clock.
    time_until_next_step = m.opt.timestep - (time.time() - step_start)
    if time_until_next_step > 0:
      time.sleep(time_until_next_step)

# print('test')
# renderer = mujoco.Renderer(model)
# mujoco.mj_forward(model, data)
# renderer.update_scene(data)
# media.show_image(renderer.render())


Installing mediapy:


In [2]:
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/50_mesa.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

/tmp/ipykernel_31000/3780323363.py:1: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.util


Tue Feb 20 15:47:46 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.40.06              Driver Version: 551.23         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1070        On  |   00000000:26:00.0  On |                  N/A |
|  0%   54C    P0             33W /  166W |    1639MiB /   8192MiB |     11%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
import glfw
import mujoco

# GLFW callback functions
def on_mouse_press(window, button, action, mods):
    global dragging, mouse_x, mouse_y, attachment_point
    if button == glfw.MOUSE_BUTTON_LEFT and action == glfw.PRESS:
        # Get mouse click position and identify attachment point on robot
        mouse_x, mouse_y = glfw.get_cursor_pos(window)
        # Replace this with your logic to identify the clicked point on the robot's body (e.g., using raycasting)
        attachment_point = None  # Placeholder, replace with actual attachment point
        dragging = True

def on_mouse_release(window, button, action, mods):
    global dragging
    if button == glfw.MOUSE_BUTTON_LEFT and action == glfw.RELEASE:
        dragging = False

def on_mouse_motion(window, xpos, ypos):
    global dragging, mouse_x, mouse_y, force

    if dragging:
        # Calculate mouse movement relative to click position
        dx = xpos - mouse_x
        dy = ypos - mouse_y
        # Convert movement to force (replace with your desired force calculation)
        force = np.array([dx, dy, 0])  # Adjust the force based on your needs
        # Update mouse position for next movement calculation
        mouse_x, mouse_y = xpos, ypos


# MuJoCo simulation setup

cam = mujoco.MjvCamera()
opt = mujoco.MjvOption()



# GLFW initialization

glfw.init()
window = glfw.create_window(800, 600, "MuJoCo with GLFW", None, None)
glfw.set_cursor_pos_callback(window, on_mouse_motion)
glfw.set_mouse_button_callback(window, on_mouse_press)
glfw.set_mouse_button_callback(window, on_mouse_release)

# Simulation loop
dragging = False
mouse_x, mouse_y = None, None
attachment_point = None
force = np.zeros(3)  # Initialize force to zero

scene = mujoco.MjvScene(model, maxgeom=10000)
context = mujoco.MjrContext(model, mujoco.mjtFontScale.mjFONTSCALE_150.value)

while not glfw.window_should_close(window):
    # Render MuJoCo scene
    simstart = data.time

    while(data.time - simstart < 1.0/60.0):
        mujoco.mj_step(model, data)

    viewport = mujoco.MjrRect(0,0,0,0)
    glfw.get_framebuffer_size(window)
    
    mujoco.mjv_updateScene(model, data, opt, None, cam, 0, scene)
    mujoco.mjr_render(viewport, scene, context)

    glfw.swap_buffers(window)
    glfw.poll_events()

glfw.terminate()


FatalError: Default framebuffer is not complete, error 0x0

In [11]:
import mujoco as mj

model_dir = Path("/home/anton/master/mujoco_menagerie/universal_robots_ur5e")
model_xml = model_dir / "scene.xml"


xml = """
<mujoco>
  <worldbody>
    <light name="top" pos="0 0 1"/>
    <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_path(str(model_xml))
data = mujoco.MjData(model)

m = model
d = data

max_width = 100
max_height = 100
# ctx = mj.GLContext(max_width, max_height)
# ctx.make_current()

cam = mj.MjvCamera()
opt = mj.MjvOption()

mj.glfw.glfw.init()
window = mj.glfw.glfw.create_window(1200, 900, "Demo", None, None)
mj.glfw.glfw.make_context_current(window)
mj.glfw.glfw.swap_interval(1)

mj.mjv_defaultCamera(cam)
mj.mjv_defaultOption(opt)

scene = mj.MjvScene(model, maxgeom=10000)
context = mj.MjrContext(model, mj.mjtFontScale.mjFONTSCALE_150.value)

while not mj.glfw.glfw.window_should_close(window):
    simstart = data.time

    while (data.time - simstart < 1.0/60.0):
        mj.mj_step(model, data)

    # viewport = mj.MjrRect(0, 0, 0, 0)
    # mj.glfw.glfw.get_framebuffer_size(window)
    viewport = mj.MjrRect(0, 0, 1200, 900)
    # mj.mjv_updateScene(model, data, opt, None, cam, 0, scene)
    mj.mjv_updateScene(model, data, opt, None, cam, mj.mjtCatBit.mjCAT_ALL.value, scene)
    mj.mjr_render(viewport, scene, context)

    mj.glfw.glfw.swap_buffers(window)
    mj.glfw.glfw.poll_events()

mj.glfw.glfw.terminate()

KeyboardInterrupt: 